In [1]:
import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations

psi4.core.set_output_file("h2o2.out")

h2o2 = psi4.geometry("""0 1
O        0.000000    0.695000   -0.092486
O       -0.000000   -0.695000   -0.092486
H       -0.388142    0.895249    0.739888
H        0.388142   -0.895249    0.739888
symmetry c1
""", name="H2O2")

psi4.set_options({
    'save_jk': True,
})

e, wfn = psi4.energy("HF/cc-pvdz", return_wfn=True, molecule=h2o2)
res = tdscf_excitations(wfn, states=10)

In [2]:
!ls

h2o2.out     LAB03-1.ipynb  LAB04.ipynb  TD.ipynb
LAB02.ipynb  LAB03-2.ipynb  readme


In [3]:
!cat h2o2.out


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on nohostname
*** at Tue May 11 04:03:23 2021

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-2 entry O          line   198 file /srv/conda/envs/notebook/share/psi4/basis/cc-pvdz.gbs 
    atoms 3-4 entry H          line    22 file /srv/conda/envs/notebook/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z          

The res variable contains all results from the TDSCF calculation. It is a list, with one element per computed root.
https://www.youtube.com/watch?v=MmG-g3SEFbk


In [4]:
for k, v in res[0].items():
    print(f"{k} = {v}")

EXCITATION ENERGY = 0.2694455039861563
ELECTRIC DIPOLE TRANSITION MOMENT (LEN) = [3.21003613e-02 9.16566425e-02 3.62341184e-14]
OSCILLATOR STRENGTH (LEN) = 0.0016941607552637397
ELECTRIC DIPOLE TRANSITION MOMENT (VEL) = [-5.07209011e-02 -2.69888896e-02 -2.73568338e-14]
OSCILLATOR STRENGTH (VEL) = 0.008167415242129586
MAGNETIC DIPOLE TRANSITION MOMENT = [-7.47275721e-02  5.36865386e-03 -4.20577561e-13]
ROTATORY STRENGTH (LEN) = -0.0019067092744119922
ROTATORY STRENGTH (VEL) = -0.013529102315423317
SYMMETRY = A
SPIN = singlet
RIGHT EIGENVECTOR ALPHA = <psi4.core.Matrix object at 0x7f87b8f26170>
LEFT EIGENVECTOR ALPHA = <psi4.core.Matrix object at 0x7f87b8f24a70>
RIGHT EIGENVECTOR BETA = <psi4.core.Matrix object at 0x7f87b8f26170>
LEFT EIGENVECTOR BETA = <psi4.core.Matrix object at 0x7f87b8f24a70>


In [10]:
import numpy as np

# get dipole moment integrals
mints = psi4.core.MintsHelper(wfn.basisset())
C_L = wfn.Ca_subset("SO", "OCC")
C_R = wfn.Ca_subset("SO", "VIR")
dipole = [psi4.core.triplet(C_L, x, C_R, True, False, False) for x in mints.so_dipole()]

for x in res:
    # Expression in Pedersen, T. B.; Hansen, A. E. Chem. Phys. Lett. 1995, 246, 1
    edtm = np.sqrt(2) * np.array([x["RIGHT EIGENVECTOR ALPHA"].vector_dot(u) for u in dipole])
    f = 2/3 * x["EXCITATION ENERGY"] * np.sum(edtm**2)
    np.testing.assert_allclose(edtm, x["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"])
    np.testing.assert_allclose(f, x["OSCILLATOR STRENGTH (LEN)"])

In [11]:
import numpy as np

# get poles and residues to plot OPA and ECD spectra
poles = [r["EXCITATION ENERGY"] for r in res]
opa_residues = [np.linalg.norm(r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"])**2 for r in res]
ecd_residues = [r["ROTATORY STRENGTH (LEN)"] for r in res]

In [12]:
from psi4.driver.p4util import spectrum

opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")

ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")

In [8]:
opa_spectrum["sticks"]

{'poles': array([169.10043718, 144.49150433, 127.41476805, 121.43070248,
        105.34273777,  97.04279195,  92.63433024,  90.39509515,
         86.01607765,  81.24291637]),
 'residues': array([1.34493771e+00, 4.83331106e-07, 5.67951313e+00, 6.94909449e+01,
        2.30986386e+03, 1.10128056e+03, 6.76294537e-01, 5.20154692e+04,
        1.52070099e+03, 5.56763023e+02])}

In [9]:
!pip install altair vega_datasets

     |████████████████████████████████| 210 kB 3.9 MB/s eta 0:00:01


In [13]:
# define plot_spectrum using altair

from typing import Tuple, Dict

import pandas as pd
import altair as alt

def plot_spectrum(data: Dict,
               *,
               title: str = "",
               x_title: Tuple[str, str] = ("ω", "au"),
               y_title: Tuple[str, str] = ("ε", "L⋅mol⁻¹⋅cm⁻¹"),
               offset: int = 0):
    hover = alt.selection_single(
      fields=["x"],
      nearest=True,
      on="mouseover",
      empty="none",
      clear="mouseout"
    )

    s1 = pd.DataFrame(data["convolution"])
    lines = alt.Chart(s1).mark_line(size=1.5).encode(
       x=alt.X("x", axis=alt.Axis(title=f"{x_title[0]} [{x_title[1]}]", offset=offset)),
       y=alt.Y("y", axis=alt.Axis(title=f"{y_title[0]} [{y_title[1]}]")),
       )

    points = lines.transform_filter(hover).mark_circle()

    tooltips = alt.Chart(s1).mark_rule().encode(
      x='x:Q',
      opacity=alt.condition(hover, alt.value(0.3), alt.value(0)),
      tooltip=[alt.Tooltip("x:Q", format=".4f", title=f"{x_title[0]}"), alt.Tooltip("y:Q", format=".1f", title=f"{y_title[0]}")]
      ).add_selection(
        hover
        )

    s2 = pd.DataFrame(data["sticks"])
    sticks = alt.Chart(s2).mark_bar(size=2, opacity=0.2, color="red").encode(
        x="poles:Q",
        y="residues:Q",
        )

    # Put the layers into a chart and bind the data
    plot = alt.layer(
      lines, points, tooltips, sticks,
      ).properties(
        title=title,
        )

    return plot

In [14]:
#from altair_spectrum import plot_spectrum

opa_plot = plot_spectrum(opa_spectrum,
                         title="OPA (Gaussian broadening)",
                         x_title=("λ", "nm"))

ecd_plot = plot_spectrum(ecd_spectrum,
                         title="ECD (Gaussian broadening)",
                         x_title=("λ", "nm"),
                         y_title=("Δε", "L⋅mol⁻¹⋅cm⁻¹"))

opa_plot & ecd_plot

alt.VConcatChart(...)

In [17]:
import numpy as np
import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from psi4.driver.p4util import spectrum

psi4.core.set_output_file("moxy.out")

moxy = psi4.geometry("""0 1
C  0.152133 -0.035800  0.485797
C -1.039475  0.615938 -0.061249
C  1.507144  0.097806 -0.148460
O -0.828215 -0.788248 -0.239431
H  0.153725 -0.249258  1.552136
H -1.863178  0.881921  0.593333
H -0.949807  1.214210 -0.962771
H  2.076806 -0.826189 -0.036671
H  2.074465  0.901788  0.325106
H  1.414895  0.315852 -1.212218
""", name="(S)-methyloxirane")

psi4.set_options({
    'save_jk': True,
})

e, wfn = psi4.energy("HF/cc-pvdz", return_wfn=True, molecule=moxy)
res = tdscf_excitations(wfn, states=8, triplets="also")

#print(res)

# get poles and residues to plot OPA and ECD spectra
#poles = [r["EXCITATION ENERGY"] for r in res]
#opa_residues = [np.linalg.norm(r["LENGTH-GAUGE ELECTRIC DIPOLE TRANSITION MOMENT"])**2 for r in res]
#ecd_residues = [r["LENGTH-GAUGE ROTATORY STRENGTH"] for r in res]

poles = [r["EXCITATION ENERGY"] for r in res]
opa_residues = [np.linalg.norm(r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"])**2 for r in res]
ecd_residues = [r["ROTATORY STRENGTH (LEN)"] for r in res]

#opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")
#ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")


opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")
ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")

In [18]:
#from altair_spectrum import plot_spectrum

opa_plot = plot_spectrum(opa_spectrum,
                         title="OPA (Gaussian broadening)",
                         x_title=("λ", "nm"))

ecd_plot = plot_spectrum(ecd_spectrum,
                         title="ECD (Gaussian broadening)",
                         x_title=("λ", "nm"),
                         y_title=("Δε", "L⋅mol⁻¹⋅cm⁻¹"))

opa_plot & ecd_plot

alt.VConcatChart(...)

In [9]:
!cat moxy.out


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on nohostname
*** at Tue May 11 03:04:35 2021

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-3  entry C          line   138 file /srv/conda/envs/notebook/share/psi4/basis/cc-pvdz.gbs 
    atoms 4    entry O          line   198 file /srv/conda/envs/notebook/share/psi4/basis/cc-pvdz.gbs 
    atoms 5-10 entry H          line    22 file /srv/conda/envs/notebook/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C1

    Geometry (in Angstrom), cha